In [33]:
import os
import pandas as pd
import re
import time
from bs4 import BeautifulSoup

In [4]:
COUNTRY = 'UK'
HTML_FILE_PATH = '/home/jack/data/pvoutput.org/raw'

In [212]:
def process_system_size_col(soup):
    pv_system_size_col = soup.find_all('a', href=re.compile('display\.jsp\?sid='))
    
    metadata = []
    for row in pv_system_size_col:
        metadata_for_row = {}
        # Get system ID
        href = row.attrs['href']
        p = re.compile('^display\.jsp\?sid=(\d+)$')
        href_match = p.match(href)
        metadata_for_row['system_id'] = href_match.group(1)
        
        # Process title (lots of metadata in here!)
        title, title_meta = row.attrs['title'].split('|')
        
        # Name and capacity
        p = re.compile('(.*) (\d+\.\d+kW)')
        title_match = p.match(title)
        metadata_for_row['system_name'] = title_match.group(1)
        metadata_for_row['system_capacity'] = title_match.group(2)
        
        # Other key-value pairs:
        key_value = title_meta.split('<br/>')
        key_value_dict = {}
        for line in key_value:
            key_value_split = line.split(':')
            key = key_value_split[0].strip()
            # Some values have a colon(!)
            value = ':'.join(key_value_split[1:]).strip()
            key_value_dict[key] = value
        metadata_for_row.update(key_value_dict)
        
        # Some cleaning
        # Remove <img ...> from Location
        location = metadata_for_row['Location']
        p = re.compile('(<img .*\>)?(.*)')
        img_groups = p.search(location).groups()
        if img_groups[0] is not None:
            metadata_for_row['Location'] = img_groups[1].strip()
        
        metadata.append(metadata_for_row)
        
    data = pd.DataFrame(metadata).set_index('system_id')
    return data


def process_output_col(soup, index=None):
    outputs_col = soup.find_all(text=re.compile('\d Days'))
    return pd.Series(outputs_col, name='Outputs', index=index)


def process_generation_and_average_cols(soup, index=None):
    generation_and_average_cols = soup.find_all(text=re.compile('\d[Mk]Wh$'))
    generation_col = generation_and_average_cols[0::2]
    average_col = generation_and_average_cols[1::2]
    return pd.DataFrame({
        'Generation': generation_col,
        'Average': average_col
    }, index=index)

def process_efficiency_col(soup, index=None):
    efficiency_col = soup.find_all(text=re.compile('\dkWh/kW'))
    return pd.Series(efficiency_col, name='Efficiency', index=index)

In [209]:
def process_all_metadata(soup):
    pv_system_size_metadata = process_system_size_col(soup)
    index = pv_system_size_metadata.index

    pv_systems_metadata = [
        pv_system_size_metadata,
        process_output_col(soup, index),
        process_generation_and_average_cols(soup, index),
        process_efficiency_col(soup, index)
    ]

    return pd.concat(pv_systems_metadata, axis='columns')

In [210]:
def get_soup(page):
    filename = os.path.join(HTML_FILE_PATH, 'pvoutput_{}_page_{:03d}.html'.format(COUNTRY, page))
    with open(filename, mode='r') as fh:
        soup = BeautifulSoup(fh, 'html.parser')
    return soup

In [214]:
all_metadata = []
for page in range(128):
    print('\r', page, end='', flush=True)
    soup = get_soup(page)
    all_metadata.append(process_all_metadata(soup))

 127

In [217]:
all_metadata_df = pd.concat(all_metadata)

In [220]:
len(all_metadata_df)

2559

In [221]:
all_metadata_df.head()

,Array Tilt,Inverter,Location,Orientation,Panels,Shade,system_capacity,system_name,Outputs,Generation,Average,Efficiency
system_id,,,,,,,,,,,,
26965,11.0°,Enphase M215,United Kingdom CV47,South 0.0°,336x250W Q Cells Pro-G3 250,Low,84.000kW,mfl_phc,"1,813 Days",399.238MWh,220.208kWh,2.654kWh/kW
24768,NaN,aurora trio,United Kingdom OX7,South 180.0°,1050x240W qidong,No,252.000kW,mfl_qfa,446 Days,307.029MWh,688.405kWh,2.732kWh/kW
11542,15.0°,SMA TRI-Power 1700 TL,United Kingdom PE11,South 180.0°,200x250W Emmvee ES 230 M60 B,No,50.000kW,Wray Farms Solar System,"2,437 Days",293.684MWh,120.510kWh,2.477kWh/kW
66991,40.0°,Unknown,United Kingdom HR8,South 180.0° / South West 225.0°,152x325W + 80x325W Unknown,No,75.400kW,Ledbury Community Hospital,"1,434 Days",279.902MWh,195.190kWh,2.603kWh/kW
5116,15.0°,Fronius CL36; Datamanager 2.0,United Kingdom OX7,South 180.0°,182x235W Q.Base-G2 235,No,42.770kW,mfl_scf,"2,538 Days",267.470MWh,105.386kWh,2.493kWh/kW


In [ ]:
all_metadata_df.index = pd.to_numeric(all_metadata_df.index)

In [238]:
# Massage text data
# Array tilt
array_tilt = all_metadata_df['Array Tilt'].str.replace('°', '').str.split('/', expand=True)
array_tilt.rename({0: 'system_1_tilt_degrees', 1: 'system_2_tilt_degrees'}, axis='columns', inplace=True)
array_tilt = array_tilt.astype(float)

In [ ]:
all_metadata_df = pd.concat([all_metadata_df, array_tilt], axis='columns')

In [252]:
# Outputs
all_metadata_df['Outputs'] = pd.to_timedelta(all_metadata_df['Outputs'])

In [290]:
# Do all system capacity columns include kW?  Yes they do!
all_metadata_df['system_capacity'].str.contains('kW').sum() == len(all_metadata_df)

True

In [270]:
all_metadata_df['system_capacity_kw'] = pd.to_numeric(
    all_metadata_df['system_capacity'].str.replace('kW', ''))

In [284]:
all_metadata_df['efficiency_kWh_per_kW'] = pd.to_numeric(
    all_metadata_df['Efficiency'].str.strip('kWh/kW'))

In [299]:
all_metadata_df.to_hdf(
    '/home/jack/data/pvoutput.org/processed/UK_PV_listing_metadata.hdf', 
    key='metadata',
    mode='w',
    format='table',
    complevel=9
)